In [ ]:
from sklearn.linear_model import LogisticRegression

# Initialize and train model
log_reg = LogisticRegression()
# log_reg.fit(X_train, y_train)

# # Predict and evaluate
# y_pred = log_reg.predict(X_test)
# log_reg_acc = accuracy_score(y_test, y_pred)
# print(f"Logistic Regression Accuracy: {log_reg_acc:.4f}")


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Initialize and train model
dt = DecisionTreeClassifier()
# dt.fit(X_train, y_train)

# # Predict and evaluate
# y_pred = dt.predict(X_test)
# dt_acc = accuracy_score(y_test, y_pred)
# print(f"Decision Tree Accuracy: {dt_acc:.4f}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train model
rf = RandomForestClassifier(n_estimators=100)
# rf.fit(X_train, y_train)

# # Predict and evaluate
# y_pred = rf.predict(X_test)
# rf_acc = accuracy_score(y_test, y_pred)
# print(f"Random Forest Accuracy: {rf_acc:.4f}")

In [ ]:
from xgboost import XGBClassifier

# Initialize and train model
xgb = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
# xgb.fit(X_train, y_train)

# # Predict and evaluate
# y_pred = xgb.predict(X_test)
# xgb_acc = accuracy_score(y_test, y_pred)
# print(f"XGBoost Accuracy: {xgb_acc:.4f}")


In [ ]:
from sklearn.svm import SVC

# Initialize and train model
svm = SVC(kernel='rbf', probability=True)
# svm.fit(X_train, y_train)

# # Predict and evaluate
# y_pred = svm.predict(X_test)
# svm_acc = accuracy_score(y_test, y_pred)
# print(f"SVM Accuracy: {svm_acc:.4f}")


In [ ]:
from sklearn.neural_network import MLPClassifier

# Initialize and train model
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)
# mlp.fit(X_train, y_train)

# # Predict and evaluate
# y_pred = mlp.predict(X_test)
# mlp_acc = accuracy_score(y_test, y_pred)
# print(f"MLP Accuracy: {mlp_acc:.4f}")


In [ ]:
from sklearn.naive_bayes import GaussianNB

# Initialize and train model
nb = GaussianNB()
# nb.fit(X_train, y_train)

# # Predict and evaluate
# y_pred = nb.predict(X_test)
# nb_acc = accuracy_score(y_test, y_pred)
# print(f"Naïve Bayes Accuracy: {nb_acc:.4f}")


In [ ]:
import wittgenstein as lw

# Initialize and train model
ripper = lw.RIPPER()


# ripper.fit(X_train, y_train)

# # Predict and evaluate
# y_pred = ripper.predict(X_test)
# ripper_acc = accuracy_score(y_test, y_pred)
# print(f"Decision Rules (RIPPER) Accuracy: {ripper_acc:.4f}")